In [1]:
## chargement de BigQuery
%load_ext google.cloud.bigquery

## Requête
from pandas.io import gbq

In [2]:
data = """
/* -- Création de la table -- */

with visitIdPers as (

/* Récupération de l'univers visité */
select A.date, A.visitId, A.fullVisitorid, environnement, cd.value as idPersonne, B.app, device.isMobile as mobile
from `bouygues-telecom.93892367.ga_sessions_*` as A, unnest(hits) as hits, unnest(hits.customDimensions) as cd

/* Récupération de la variable APP */
inner join (select date, visitId, fullVisitorid, cd.value as app 
from `bouygues-telecom.93892367.ga_sessions_*`, unnest(hits) as hits, unnest(hits.customDimensions) as cd
where _table_suffix = '20180925'
and cd.index = 2

) as B
on A.date = B.date
and A.visitId = B.visitId
and A.fullVisitorid = B.fullVisitorid


/* Récupération de l'univers */
inner join (select date, visitId, fullVisitorid, case when regexp_contains (hits.page.pagePath, 'Espace_client') then 'Espace Client' else cd.value end as environnement
from `bouygues-telecom.93892367.ga_sessions_*`, unnest(hits) as hits, unnest(hits.customDimensions) as cd
where _table_suffix = '20180925'
and cd.index = 15

) as C
on A.date = C.date
and A.visitId = C.visitId
and A.fullVisitorid = C.fullVisitorid


where _table_suffix = '20180925'
and cd.index = 9
and  regexp_contains (cd.value , '800003539278') 

)


/* -- Requêtage de la table -- */

select date, idPersonne, environnement, app, mobile, count(distinct concat(date, cast(visitId as string), fullVisitorid)) as nb_vist from visitIdPers 
group by 1,2,3,4,5
"""

## sauvegarde de la requête en dataFrame 
dataBQ = gbq.read_gbq(data, dialect="standard")

In [3]:
## Affichage du dataFrame 
dataBQ.head()

,date,idPersonne,environnement,app,mobile,nb_vist
0,20180925,800003539278,Boutique web,web,False,1
1,20180925,800003539278,Boutique mobile,web,False,3
2,20180925,800003539278,Espace Client,web,False,3
3,20180925,800003539278,Espace Client,Appli_iOS,True,1


In [4]:
dataBQ.count()

date             4
idPersonne       4
environnement    4
app              4
mobile           4
nb_vist          4
dtype: int64

In [5]:
##test2